# Langchain crash course

Langchain is a framework allowing us build applications with LLMs. When we use ChatGPT (which is not an LLM but an application), it makes call to OpenAI API which interacts with an LLM (like GPT-3.5/ GPT-4) to get the answer.

environment = `rag_env`

In [1]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

## LLMs

In [2]:
import langchain
print(langchain.__version__)

0.3.13


In [11]:
## Deprecated 
# from langchain.llms import OpenAI # Other LLM providers: OpenAI, Cohere, Hugging Face 
# name = llm.predict('Who is Narendra Modi') # Deprecated: predict to invoke 

from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9) # temperature controls creativity


In [ ]:

name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)

<span style="color:yellow">default model is "gpt-3.5-turbo-instruct"</span>

There is a list of openai models that we could use from list shown at https://api.python.langchain.com/en/latest/_modules/langchain_openai/llms/base.html#OpenAI

The syntax would be 
`llm=OpenAI(model='gpt-4o', temperature=0.2)`




## Prompt Templates

We don't want to keep writing the prompt for every query. In those case, we can use Prompt Templates.

In [12]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

print(llm.invoke(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.

"Bellissimo Bistro"


Let's vary 2 (cusisine and place) variables instead of 1 (cuisine)

In [11]:
prompt_template_name_place = PromptTemplate(
    input_variables=['cuisine', 'place'],
    template="I want to open a restaurant for {cuisine} food in {place} country. Suggest a fancy name and a city for this."
)

p = prompt_template_name_place.format(cuisine="Italian", place='India')
print(p)

print(llm.invoke(p))

I want to open a restaurant for Italian food in India country. Suggest a fancy name and a city for this.


"Fresco Elegante" in Mumbai, India.


## Chains

What is we want to do multiple steps like decide a name of restaurant, a place for it and then create a menu for the dishes. Then, an easier way to handle is by using Chains. Here, ouptput of first step is input to second and output of second is input to third and so on. 

If we are using a `SimpleSequentialChain` then only the final output (last step's output) is returned and rest other intermediate outputs are not.

So, if we need to get the intermediate outputs as well. We could use `SequentialChain`

What other types of chain exists <span style="color:red">???</span>

In [ ]:
from langchain.chains import LLMChain


In [ ]:


chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.invoke("Mexican")

## Deprecated 
# chain.run -> chain.invoke

<span style = "color: yellow"> If `verbose=True`, we will be able to see what's happening unde the hood and not just the final output.</span>

In [15]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.invoke("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


{'cuisine': 'Mexican', 'text': '\n\nCasa de Sabor (House of Flavor)'}

Suppose we want to create the name of restaurant and also create the menu on the basis of the name. So, let's create 2 chains - 1 for name and 1 for menu.

In [16]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)


Now we can see that both these chains are not connected. Because if we want to create menu, we need name. Below I am providing the name myself.

In [18]:
food_items_chain.invoke('Maharaja Spice')

{'restaurant_name': 'Maharaja Spice',
 'text': '\n\n1. Tandoori Chicken: Tender chicken marinated in a blend of spices and cooked in a clay oven.\n\n2. Butter Chicken: Boneless chicken cooked in a rich and creamy tomato-based sauce.\n\n3. Lamb Rogan Josh: A classic dish of tender lamb cooked in a spicy tomato and onion-based gravy.\n\n4. Vegetable Biryani: Fragrant basmati rice cooked with mixed vegetables, herbs, and spices.\n\n5. Palak Paneer: Soft cubes of paneer (Indian cottage cheese) cooked in a creamy spinach sauce.\n\n6. Dal Makhani: A popular lentil dish made with black lentils, kidney beans, and aromatic spices.\n\n7. Malai Kofta: Vegetable dumplings made with potatoes, carrots, and peas, served in a creamy cashew-based sauce.\n\n8. Aloo Gobi: A delicious combination of potatoes and cauliflower cooked with onions, tomatoes, and spices.\n\n9. Chicken Tikka Masala: Succulent pieces of chicken cooked in a flavorful tomato and onion-based sauce.\n\n10. Garlic Naan: Soft and fluff

Now, we don't want to pass the name by ourselves but get the name from 1st chain and pass that into 2nd chain. How can we do that?

#### Simple Sequential Chain

In [19]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain]) # sequence/order of the chains matter

content = chain.invoke("Indian")
print(content)

{'input': 'Indian', 'output': '\n1. Spicy Chicken Tikka Masala\n2. Lamb Vindaloo\n3. Vegetable Samosas\n4. Tandoori Shrimp Skewers\n5. Aloo Gobi (spiced cauliflower and potatoes)\n6. Chana Masala (spiced chickpea curry)\n7. Butter Chicken\n8. Beef Korma\n9. Palak Paneer (spinach and cheese curry)\n10. Dal Makhani (creamy lentil dish)\n11. Tandoori Naan bread\n12. Mango Lassi (a refreshing yogurt drink)\n13. Spicy Chicken Biryani\n14. Mixed Vegetable Pakoras\n15. Lamb Rogan Josh (spicy lamb curry)\n16. Achari Chicken (pickled spiced chicken)\n17. Baingan Bharta (spiced eggplant dish)\n18. Malai Kofta (vegetable and cheese dumplings in a creamy sauce)\n19. Chicken Jalfrezi (spicy stir-fry with vegetables)\n20. Gulab Jamun (sweet fried dough balls in syrup)'}


Here we can see that even if we got the menu as output (output of last chain), we didn't get the restaurant name (output of chain 1). To get intermediate outputs, use `SequentialChain`

#### Sequential Chain

In [20]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [24]:
name_chain.invoke('Indian')

{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Spice Palace: A Taste of India"'}

In [25]:
llm = OpenAI(temperature=0.7)

prompt_template_name_place = PromptTemplate(
    input_variables =['cuisine', 'place'],
    template = "I want to open a restaurant for {cuisine} food inside {place}. Suggest a fancy name and a place for this."
)

name_place_chain =LLMChain(llm=llm, prompt=prompt_template_name_place, output_key="restaurant_name")

In [33]:
name_place_chain.invoke({'cuisine': 'Indo-chinese', 'place': 'Mumbai'})

{'cuisine': 'Indo-chinese',
 'place': 'Mumbai',
 'restaurant_name': '\n\nName: "Spice Fusion Bistro"\n\nLocation: Bandra West, Mumbai'}

In [34]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [35]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [37]:
chain.invoke({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Spice Palace"',
 'menu_items': '\n\n1. Chicken Tikka Masala \n2. Vegetable Samosas \n3. Lamb Vindaloo \n4. Chana Masala \n5. Palak Paneer \n6. Tandoori Chicken \n7. Naan Bread \n8. Mango Lassi \n9. Aloo Gobi \n10. Butter Chicken \n11. Vegetable Biryani \n12. Papadum \n13. Dal Makhani \n14. Chicken Korma \n15. Gulab Jamun (dessert)'}

## Agents

Agents  - will have access to external tools. Example: Expedia, mathematics function etc.

Tools list: https://python.langchain.com/v0.1/docs/integrations/tools/

In [7]:
# make sure yoy have installed this package: pip install google-search-results
from secret_key import serpapi_key
os.environ['SERPAPI_API_KEY'] = serpapi_key

#### serpapi and llm-math tool

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools
# from langchain.llms import OpenAI # Deprecated
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.invoke("What was the GDP of US in 2022 plus 5 trillion?")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to solve this math problem.
Action: Calculator
Action Input: 2022 + 5 trillion
Observation: Answer: 5000000002022
Thought: This doesn't seem right, I should try a different approach.
Action: Calculator
Action Input: 2022 * 5 trillion
Observation: Answer: 10110000000000000
Thought: This answer seems more accurate, but I should double check with a different tool.
Action: Search
Action Input: "GDP of US in 2022"
Observation: ["The U.S. is a country of 50 states covering a vast swath of North America, with Alaska in the northwest and Hawaii extending the nation’s presence into the Pacific Ocean. Major Atlantic Coast cities are New York, a global finance and culture center, and capital Washington, DC. Midwestern metropolis Chicago is known for influential architecture and on the west coast, Los Angeles' Hollywood is famed for filmmaking. ― Google", 'United States type: Country in North America.', 'United States entity

{'input': 'What was the GDP of US in 2022 plus 5 trillion?',
 'output': 'The GDP of the US in 2022 plus 5 trillion is approximately $30.46 trillion.'}

####### Peculiar Agent Response Snapshot. See how it rectifies itself ########


![alt text](peculiar_agent_response.png "Title")

#### Wikipedia and llm-math tool

In [10]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.invoke("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: wikipedia
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk  (; born June 28, 1971) is a businessman known for his key roles in the space company SpaceX and the automotive company Tesla, Inc. His other involvements include ownership of X Corp., the company that operates the social media platform X (formerly Twitter), and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. Musk is the wealthiest individual in the world; as of December 2024, Forbes estimates his net worth to be US$432 billion.
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at the age of 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later tra

{'input': 'When was Elon musk born? What is his age right now in 2023?',
 'output': 'Elon Musk is currently 52 years old in 2023.'}

## Memory

Memory means remembering old conversation. But remembering all the old conversation is also not required. So, we could limit from practical pov where it becomes costly to store and retokenize & reprocess all the old queries in order to generate a response. So, in this section, we will see how we can limit it to say last 5 queries or last 2 queries and so on, depending upon our needs.

In [16]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'text': '\n\n"La Fiesta Mexicana"'}


In [17]:
name = chain.invoke("Indian")
print(name)

{'cuisine': 'Indian', 'text': '\n\n"Maharaja\'s Palace"'}


In [20]:
chain.memory

<span style='color:yellow'> Nothing gets printed here because `chain.memory` is `None`

In [21]:
type(chain.memory)

NoneType

To add memory use `ConversationBufferMemory`

#### ConversationBufferMemory

In [24]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'history': '', 'text': '\n\n"El Sabroso Cantina"'}


In [25]:
name = chain.invoke("Arabic")
print(name)

{'cuisine': 'Arabic', 'history': 'Human: Mexican\nAI: \n\n"El Sabroso Cantina"', 'text': '\n\n"Al-Quds Oasis"'}


As one can notice above, we start to see the `history`

In [26]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabroso Cantina"
Human: Arabic
AI: 

"Al-Quds Oasis"


We can use these Q. Ans. pair later for finetuning/ creating a db/ monitoring after storing them

 <span style='color:yellow'> But the problem with current implmentation is that it will store all the interactions in memory in order to answer a new one. This is also true for the `ConversationChain` shown below.

#### ConversationChain

In [28]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template) # default template

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [29]:
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': " The first cricket world cup was won by West Indies in 1975. The final match was played between West Indies and Australia at Lord's Cricket Ground in London. West Indies won by 17 runs, with Clive Lloyd as their captain. Interestingly, the first match of the tournament was played between England and India at Lord's as well, with England winning by 202 runs."}

In [30]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by West Indies in 1975. The final match was played between West Indies and Australia at Lord's Cricket Ground in London. West Indies won by 17 runs, with Clive Lloyd as their captain. Interestingly, the first match of the tournament was played between England and India at Lord's as well, with England winning by 202 runs.",
 'response': '  5+5 is equal to 10. This is a basic arithmetic equation and the answer is derived from the fact that 5+5 equals to 10.'}

In [31]:
convo.invoke("Who was the captain ofthe winning team?")

{'input': 'Who was the captain ofthe winning team?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by West Indies in 1975. The final match was played between West Indies and Australia at Lord's Cricket Ground in London. West Indies won by 17 runs, with Clive Lloyd as their captain. Interestingly, the first match of the tournament was played between England and India at Lord's as well, with England winning by 202 runs.\nHuman: How much is 5+5?\nAI:   5+5 is equal to 10. This is a basic arithmetic equation and the answer is derived from the fact that 5+5 equals to 10.",
 'response': ' The captain of the winning team for the first cricket world cup was Clive Lloyd of West Indies.'}

In [32]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by West Indies in 1975. The final match was played between West Indies and Australia at Lord's Cricket Ground in London. West Indies won by 17 runs, with Clive Lloyd as their captain. Interestingly, the first match of the tournament was played between England and India at Lord's as well, with England winning by 202 runs.
Human: How much is 5+5?
AI:   5+5 is equal to 10. This is a basic arithmetic equation and the answer is derived from the fact that 5+5 equals to 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team for the first cricket world cup was Clive Lloyd of West Indies.


<span style='color:yellow'> Now using `ConversationBufferWindowMemory`, we can decide how many past conversations should be remembered and repassed as context to LLM to answer the new query

#### ConversationBufferWindowMemory

In [34]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1) # meaning remember only 1 last conversation and repass to LLM as context

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.invoke("Who won the first cricket world cup?")

' The first cricket world cup was won by the West Indies in 1975. It was held in England and the final match was played between the West Indies and Australia. The West Indies won by 17 runs.'

In [ ]:
convo.invoke("How much is 5+5?")

' 5+5 is equal to 10.'

In [ ]:
convo.invoke("Who was the captain of the winning team?")

' I do not have enough information to accurately answer that question. Could you provide me with more context?'

In [37]:
memory = ConversationBufferWindowMemory(k=2) # meaning remember only 1 last conversation and repass to LLM as context

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.invoke("Who won the first cricket world cup?")
convo.invoke("How much is 5+5?")
convo.invoke("Who was the captain of the winning team?")

{'input': 'Who was the captain of the winning team?',
 'history': 'Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was held in 1975 in England and was won by the West Indies team.\nHuman: How much is 5+5?\nAI:  5+5 is equal to 10.',
 'response': ' The captain of the winning team was Clive Lloyd.'}